## Global Fishing Watch Annual Product Processing

In [1]:
import os
import arcpy
from arcpy.sa import *
import time

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# Set the workspace to the geodatabase
arcpy.env.workspace = r'E:\gis_lib\hexbins\H3_Hexbins.gdb'
arcpy.env.overwriteOutput = True

p_type = 'gfw_2020_annual'
summary_raster = r'e:\analysis\GFW\2020\TIF\global_fish_watch_annual_fleet_sum_2020.crf'
gdb = r'E:\analysis\GFW\gfw_2020_summary.gdb'

# List all feature classes in the geodatabase
feature_classes = arcpy.ListFeatureClasses()
global_summary_layer = r'E:\gis_lib\general\Base_content.gdb\Global_Extent_WGS84'
global_summary_field = 'OBJECTID'
hex_summary_field = 'GRID_ID'
eez_summary_layer = r'E:\gis_lib\general\Base_content.gdb\eez_v12'
eez_summary_field = 'MRGID_ID'

### Summarize the Global Fishing Watch imagery into hexbins

In [2]:
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(gdb), os.path.basename(gdb))
    print(f"Created geodatabase: {gdb}")
else:
    print(f"Geodatabase already exists: {gdb}")

Geodatabase already exists: E:\analysis\GFW\gfw_2020_summary.gdb


In [3]:
# Print the feature classes
for fc in feature_classes:
    gdb_root = os.path.split(gdb)[0]
    fc_short = fc.replace('GenerateTessellation_', '').replace('_wgs84', '')
    print("Processing... " + fc_short)
    print(datetime.datetime.now())
    print(os.path.join(gdb, p_type + '_' + fc))
    # print(gdb_root)
    # print(fc_short)
    print("Running Zonal Statistics as Table...")
    with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=fc,
            zone_field=hex_summary_field,
            in_value_raster=summary_raster,
            out_table = os.path.join(gdb, p_type + '_' + fc_short),
            ignore_nodata="DATA",
            statistics_type="SUM",
            process_as_multidimensional="ALL_SLICES",
            percentile_values=90,
            percentile_interpolation_type="AUTO_DETECT",
            circular_calculation="ARITHMETIC",
            circular_wrap_value=360,
            out_join_layer = os.path.join(gdb_root, p_type + '_' + fc_short + '.lyr')
        )
        print("Running Pivot Table...")
        arcpy.management.PivotTable(
            in_table=os.path.join(gdb, p_type + '_' + fc_short),
            fields= hex_summary_field + ';' + "StdTime",
            pivot_field="Variable",
            value_field="SUM",
            out_table=os.path.join(gdb, p_type + '_' + fc_short + '_pivot')
        )
        print("Running Copy Features...")
        arcpy.management.CopyFeatures(
            in_features=fc,
            out_feature_class=os.path.join(gdb, p_type + '_' + fc_short + '_fc'),
            config_keyword="",
            spatial_grid_1=None,
            spatial_grid_2=None,
            spatial_grid_3=None
        )
        print("Running Join Field...")
        arcpy.management.JoinField(
            in_data=os.path.join(gdb, p_type + '_' + fc_short + '_fc'),
            in_field=hex_summary_field,
            join_table=os.path.join(gdb, p_type + '_' + fc_short + '_pivot'),
            join_field=hex_summary_field,
            fields="fishhours; hours",
            fm_option="NOT_USE_FM",
            field_mapping=None,
            index_join_fields="NO_INDEXES"
        )

Processing... h3_r6
2024-09-25 23:02:19.864724
E:\analysis\GFW\gfw_2020_summary.gdb\gfw_2020_annual_GenerateTessellation_h3_r6_wgs84
Running Zonal Statistics as Table...
Running Pivot Table...
Running Copy Features...
Running Join Field...
Processing... h3_r5
2024-09-25 23:58:05.719080
E:\analysis\GFW\gfw_2020_summary.gdb\gfw_2020_annual_GenerateTessellation_h3_r5_wgs84
Running Zonal Statistics as Table...
Running Pivot Table...
Running Copy Features...
Running Join Field...
Processing... h3_r4
2024-09-26 00:09:26.899443
E:\analysis\GFW\gfw_2020_summary.gdb\gfw_2020_annual_GenerateTessellation_h3_r4_wgs84
Running Zonal Statistics as Table...
Running Pivot Table...
Running Copy Features...
Running Join Field...
Processing... h3_r3
2024-09-26 00:12:27.014184
E:\analysis\GFW\gfw_2020_summary.gdb\gfw_2020_annual_GenerateTessellation_h3_r3_wgs84
Running Zonal Statistics as Table...
Running Pivot Table...
Running Copy Features...
Running Join Field...
Processing... h3_r2
2024-09-26 00:13:52.

### Summarize the data for the entire globe

In [4]:
with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=global_summary_layer,
        zone_field=global_summary_field,
        in_value_raster=summary_raster,
        out_table = os.path.join(gdb, p_type + '_global_summary'),
        ignore_nodata="DATA",
        statistics_type="ALL",
        process_as_multidimensional="ALL_SLICES",
        percentile_values=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360,
        out_join_layer = os.path.join(gdb, p_type + '_summary.lyr')
    )

### Summarize fishing activity for the different Exclusive Economic Zones (EEZ)

In [6]:
with arcpy.EnvManager(parallelProcessingFactor="99%", cellSize="MINOF"):
    print("Running Zonal Statistics as Table...")
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=eez_summary_layer,
        zone_field=eez_summary_field,
        in_value_raster=summary_raster,
        out_table = os.path.join(gdb, p_type + '_eez_summary'),
        ignore_nodata="DATA",
        statistics_type="ALL",
        process_as_multidimensional="ALL_SLICES",
        percentile_values=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360,
        out_join_layer = os.path.join(gdb, p_type + '_eez_summary.lyr')
    )
    print("Running Pivot Table...")
    arcpy.management.PivotTable(
        in_table=os.path.join(gdb, p_type + '_eez_summary'),
        fields= eez_summary_field + ';' + "StdTime",
        pivot_field="Variable",
        value_field="SUM",
        out_table=os.path.join(gdb, p_type + '_eez_summary_pivot')
    )
    print("Running Copy Features...")
    arcpy.management.CopyFeatures(
        in_features=eez_summary_layer,
        out_feature_class=os.path.join(gdb, p_type + '_' + '_eez_summary' + '_fc'),
        config_keyword="",
        spatial_grid_1=None,
        spatial_grid_2=None,
        spatial_grid_3=None
    )
    print("Running Join Field...")
    arcpy.management.JoinField(
        in_data=os.path.join(gdb, p_type + '_' + '_eez_summary' + '_fc'),
        in_field=eez_summary_field,
        join_table=os.path.join(gdb, p_type + '_eez_summary_pivot'),
        join_field=eez_summary_field,
        fields="fishhours; hours",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )

Running Pivot Table...
Running Copy Features...
Running Join Field...
